In [1]:
from torch import nn
import pandas as pd
import torch
from torch.autograd import Variable
import torch.utils.data as data
import numpy as np
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, matthews_corrcoef, accuracy_score
from sklearn.ensemble import ExtraTreesClassifier, AdaBoostClassifier

In [2]:
data = pd.read_pickle("wanted.pkl")

In [3]:
import pickle
with open('merged_embeddings.pkl','rb') as file:
    merged_embeddings = pickle.load(file)

In [4]:
data['embedding'] = data.apply(lambda row: merged_embeddings[row['scientificName']],axis=1)

In [5]:
len(set(data['redlistCategory']))

10

In [6]:
data = data.replace(to_replace =['Critically Endangered',
 'Endangered',
 'Extinct',
 'Extinct in the Wild',
 'Least Concern',
 'Lower Risk/conservation dependent',
 'Lower Risk/least concern',
 'Lower Risk/near threatened',
 'Near Threatened',
 'Vulnerable'], 
                 value =[0,0,0,0,1,0,1,0,0,0])

D:\Anaconda\lib\site-packages\pandas\core\internals\managers.py:1937: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


In [7]:
X = list(data['embedding'])

In [8]:
Y = list(data['redlistCategory'])

In [ ]:
# with open('X.pkl','wb') as d:
#     pickle.dump(X,d)
# with open('Y.pkl','wb') as d:
#     pickle.dump(Y,d)

In [9]:
import random
def get_batch():
    X0 = []
    X1 = []
    for x,y in zip(X_train,y_train):
        if y==0:
            X0.append(x)
        else:
            X1.append(x)
    Y0 = [0]*len(X0)
    Y1 = [1]*len(X1)
    if len(X0) <= len(X1):
        X = X0[:] + random.sample(X1,len(X0))
        Y = Y0[:] + [1]*len(X0)
    else:
        X = X1[:] + random.sample(X0,len(X1))
        Y = Y1[:] + [0]*len(X1)
    return X,Y

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
Xtrain,Ytrain = get_batch()
inp = torch.tensor(Xtrain, device = 'cuda')
out = torch.tensor(Ytrain, device = 'cuda')
inp = inp.squeeze(2)
testx = torch.tensor(X_test, device = 'cuda').squeeze(2)


In [11]:
input_size = 400
hidden_sizes = [32]
output_size = 2
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[0], output_size),
                      nn.Softmax(dim=1))
model.cuda()
model.double()
criterium = nn.CrossEntropyLoss()

In [12]:
epoch = 3000
for i in range(epoch):
    pred = model(inp)
    loss = criterium(pred, out)
    if i%100 == 0:
        print (loss.item())
        print (i)
        y_pred = model(testx)
        op = ((y_pred).argmax(1).tolist())
        print (f1_score(y_test, op))
        print (accuracy_score(y_test, op))
        print (matthews_corrcoef(y_test, op))
        print ('---------------------------------------')
    model.zero_grad()
    loss.backward()
    with torch.no_grad():
        for param in model.parameters():
            param -= 0.0001*param.grad

0.6921554454825113
0
0.7367225716282321
0.6141339137114326
0.0164135612476877
---------------------------------------
0.6921315304905762
100
0.7369982547993019
0.6141339137114326
0.014757544922468611
---------------------------------------
0.6921076737062973
200
0.7380163848701413
0.6151581103571886
0.015451493240142996
---------------------------------------
0.6920838607136391
300
0.7383088043194287
0.6152861349379081
0.01453317989987226
---------------------------------------
0.6920601439874389
400
0.7387936286883106
0.6157982332607861
0.015020012699461415
---------------------------------------
0.69203646132424
500
0.7390396659707723
0.6159262578415056
0.014377218163829494
---------------------------------------
0.6920128414763528
600
0.7390661681592905
0.6157982332607861
0.013349906308630932
---------------------------------------
0.6919892447619849
700
0.7394213224433052
0.616054282422225
0.012895355179185549
---------------------------------------
0.6919657379737504
800
0.7402326

In [15]:
test = torch.tensor(X_test, device = 'cuda').squeeze(2)
y_pred = model(test)
ypred = ((y_pred).argmax(1).tolist())

In [16]:
accuracy_score(list(ypred), y_test)

0.6265522980412239

In [17]:
X_train = np.array(X_train)
X_train = X_train.reshape(X_train.shape[0], -1)

In [18]:
X_test = np.array(X_test)
X_test = X_test.reshape(X_test.shape[0], -1)

In [19]:
clf = RandomForestClassifier(n_estimators=100, max_depth=350, n_jobs = 3,
                             random_state=0)
clf.fit(X_train, y_train)  


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=350, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=3,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [20]:
ypred = clf.predict(X_test)

In [21]:
f1_score(ypred, y_test)

0.8543768972023957

In [22]:
confusion_matrix(ypred, y_test)

array([[ 829,  373],
       [1402, 5207]], dtype=int64)

In [23]:
matthews_corrcoef(ypred, y_test)

0.3814793795503145

In [24]:
accuracy_score(ypred, y_test)

0.7727563692228908